In [ ]:
import netCDF4
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
import numpy as np
import math
import csv
import os
from tqdm import tqdm_notebook as tqdm
from keras_tqdm import TQDMNotebookCallback
from ipywidgets import interactive
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
def get_file_list(data_dir):
    return [os.path.join(data_dir, name) for name in os.listdir(data_dir)]


aq_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/aq_conc/'
emis_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/emis_data/'
met_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/met_data/'

aq_files = get_file_list(aq_dir)
emis_files = get_file_list(emis_dir)
met_files = get_file_list(met_dir)

In [ ]:
def plot_aq(aq_data, emis_data, day_of_yr, hr_of_day):
    o3ex = aq_data[hr_of_day-1][0]
    noex = aq_data[hr_of_day-1][0]

    # set canvas
    fig = plt.figure(figsize=(10,10))
    colors = [(0.33,0.33,0.33)] + [(plt.cm.jet(i)) for i in range(1,256)]
    new_map = matplotlib.colors.LinearSegmentedColormap.from_list('new_map', colors, N=64)
    
    # add the first subplot for aq(O3)
    ax = fig.add_subplot(2,1,1)
    i = ax.imshow(1000*o3ex, cmap=new_map, origin='lower',vmin=0, vmax=50, interpolation='bilinear')

    ax.set_title('Day_of_the_year = ' + str(day_of_yr) + "\n" 
                 + 'Hour_of_the_day = ' + str(hr_of_day), fontsize=16)
    ax.set_ylabel('Y_Grid', fontsize=14)
    ax.tick_params('both', labelsize=14)

    cb = fig.colorbar(i, fraction=0.046, pad=0.04)
    cb.set_label(label= 'Ozone concentration/ppb', size=14,)
    cb.ax.tick_params(labelsize=14)
    
    # add the second subplot for emis(NO)
    ax = fig.add_subplot(2,1,2)
    i = ax.imshow(noex, cmap=new_map, origin='lower', vmin=0, vmax=0.1,interpolation='bilinear')
    
    ax.set_xlabel('X_Grid', fontsize=14)
    ax.set_ylabel('Y_Grid', fontsize=14)
    ax.tick_params('both', labelsize=14)

    cb = fig.colorbar(i, fraction=0.046, pad=0.04)
    cb.set_label(label= 'NO emission rate/mol*s-1', size=14,)
    cb.ax.tick_params(labelsize=14)
    return fig
#     plt.show()

In [ ]:
Figure_map = {}
for i in range(1, 6, 1):
        for j in range(1, 25, 1):
            aq_data = netCDF4.Dataset(aq_files[i-1]) # minus one because of referring to indices of the file list
            emis_data = netCDF4.Dataset(emis_files[i-1])
            O3 = aq_data.variables['O3'][:]
            NO = emis_data.variables['NO'][:]
            
            date_hour = (i, j)
            Figure_map[date_hour] = plot_aq(O3, NO, i, j)

In [ ]:
def get_fig(i, j):
   return Figure_map.get((i,j))

interactive_plot = interact(get_fig,
                               i = widgets.IntSlider(1, 1, len(aq_files)), 
                               j = widgets.IntSlider(1, 1, 24))

interactive_plot
## Some reflections: the hashmap version of the this interactive plot is not much faster than the brutal force version,
## maybe it's becasue that in the entire process fetching the figure and display it take more time than actually plot it